# CNN and MLP Ensembling

- try CNN for images
- try MLP for features

## Preprocess Data

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv("images.csv")

# Split into features for MLP and image paths for CNN
image_paths = df['Path']
features = df.drop(columns=['Path', 'Target', 'Image_Name']).values
targets = df['Target'].values

# Encode labels
label_encoder = LabelEncoder()
targets_encoded = label_encoder.fit_transform(targets)

# Split the data
X_train_img, X_test_img, X_train_feat, X_test_feat, y_train, y_test = train_test_split(
    image_paths, features, targets_encoded, test_size=0.2
)

print(f"Image training set: {X_train_img.shape}")
print(f"Feature training set: {X_train_feat.shape}")
print(f"Target training set: {y_train.shape}")


Image training set: (388,)
Feature training set: (388, 18)
Target training set: (388,)


## CNN Model

In [89]:
import numpy as np
from tensorflow.keras.utils import img_to_array, load_img

def preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for path in image_paths:
        # Load and preprocess each image
        image = load_img(path, target_size=target_size)
        image = img_to_array(image) / 255.0  # Normalize
        images.append(image)
    return np.array(images)

# Preprocess images
X_train_cnn = preprocess_images(X_train_img)
X_test_cnn = preprocess_images(X_test_img)

print(f"CNN Input Shape: {X_train_cnn.shape}")

CNN Input Shape: (388, 128, 128, 3)


In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

cnn_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.1)

# CNN Predictions
cnn_predictions = cnn_model.predict(X_test_cnn).flatten()

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 154ms/step - accuracy: 0.5093 - loss: 1.0069 - val_accuracy: 0.4615 - val_loss: 0.7024
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 166ms/step - accuracy: 0.5577 - loss: 0.6935 - val_accuracy: 0.4872 - val_loss: 0.6882
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 224ms/step - accuracy: 0.5855 - loss: 0.6789 - val_accuracy: 0.5897 - val_loss: 0.6769
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 221ms/step - accuracy: 0.6470 - loss: 0.6403 - val_accuracy: 0.6410 - val_loss: 0.6544
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 196ms/step - accuracy: 0.7783 - loss: 0.5515 - val_accuracy: 0.6154 - val_loss: 0.7793
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 186ms/step - accuracy: 0.7615 - loss: 0.5241 - val_accuracy: 0.6410 - val_loss: 0.6700
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 199ms/step - accuracy: 0.8424 - loss: 0.4002 - val_accuracy: 0.6667 - val_loss: 0.7467
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 0.8939 - loss: 0.3186 - val_accuracy: 0.5641 - val_

In [104]:
from sklearn.metrics import accuracy_score

cnn_predictions_binary = (cnn_predictions > 0.5).astype(int) # convert to 0 or 1
cnn_accuracy = accuracy_score(y_test, cnn_predictions_binary)
print(f"CNN Test Accuracy: {cnn_accuracy:.4f}")

CNN Test Accuracy: 0.6598


## MLP Model

In [91]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [94]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# Standardize features
scaler = StandardScaler()
X_train_feat_scaled = scaler.fit_transform(X_train_feat)
X_test_feat_scaled = scaler.transform(X_test_feat)

# Define MLP model with best features
mlp_model = MLPClassifier(
    activation='relu', 
    alpha=0.001, 
    hidden_layer_sizes=(50, 30), 
    learning_rate_init=0.01, 
    solver='adam', 
    max_iter=1000, 
    random_state=42
) 
mlp_model.fit(X_train_feat_scaled, y_train)

# MLP Predictions (probabilities)
mlp_predictions = mlp_model.predict_proba(X_test_feat_scaled)[:, 1]


In [107]:
from sklearn.metrics import accuracy_score

# Get probabilities for the positive class (class 1)
mlp_predictions_proba = mlp_model.predict_proba(X_test_feat_scaled)[:, 1]

# Convert probabilities to binary class predictions using threshold (0.5)
mlp_predictions = (mlp_predictions_proba >= 0.5).astype(int)

# Calculate accuracy
mlp_accuracy = accuracy_score(y_test, mlp_predictions)
print(f"MLP Test Accuracy: {mlp_accuracy:.4f}")


MLP Test Accuracy: 0.9691


## Ensembling Part?

In [95]:
from sklearn.metrics import accuracy_score

# Simple Averaging for ensemble predictions
weights = [0.5, 0.5]  # Equal weights for CNN and MLP
combined_predictions = (weights[0] * cnn_predictions) + (weights[1] * mlp_predictions)
final_predictions = (combined_predictions > 0.5).astype(int)

# Evaluate the ensemble model's accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Ensemble Accuracy: 0.8866


In [100]:
from sklearn.metrics import accuracy_score

# CNN predictions
cnn_predictions = cnn_model.predict(X_test_cnn)  # CNN outputs probabilities
cnn_predictions = cnn_predictions.flatten()  # Ensure the shape is 1D

# MLP predictions
mlp_predictions = mlp_model.predict_proba(X_test_feat_scaled)[:, 1]  # Probabilities for class 1

# Combine predictions: weighted average (more on mlp?)
cnn_weight = 0.4
mlp_weight = 0.6
combined_predictions = (cnn_weight * cnn_predictions) + (mlp_weight * mlp_predictions)

# Step 4: threshold combined predictions to get final binary class
final_predictions = (combined_predictions > 0.5).astype(int)

# Step 5: Evaluate ensemble performance
accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Test Accuracy: {accuracy:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Ensemble Test Accuracy: 0.9278


In [101]:
import numpy as np
from sklearn.metrics import accuracy_score

# Define a grid of weights
weights = np.linspace(0, 1, 21)  # 21 values from 0.0 to 1.0

best_accuracy = 0
best_weights = (0.5, 0.5)  # Default to equal weights

for cnn_weight in weights:
    mlp_weight = 1 - cnn_weight  # Ensure weights sum to 1
    combined_predictions = (cnn_weight * cnn_predictions) + (mlp_weight * mlp_predictions)
    final_predictions = (combined_predictions > 0.5).astype(int)
    accuracy = accuracy_score(y_test, final_predictions)
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_weights = (cnn_weight, mlp_weight)

print(f"Best Weights: CNN={best_weights[0]}, MLP={best_weights[1]}")
print(f"Best Ensemble Accuracy: {best_accuracy:.4f}")


Best Weights: CNN=0.0, MLP=1.0
Best Ensemble Accuracy: 0.9691


In [102]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
best_accuracy = 0
best_weights = (0.5, 0.5)

for cnn_weight in weights:
    mlp_weight = 1 - cnn_weight
    fold_accuracies = []
    
    for train_index, test_index in kf.split(X_test):
        # Create train/test splits for cross-validation
        cnn_train, cnn_test = cnn_predictions[train_index], cnn_predictions[test_index]
        mlp_train, mlp_test = mlp_predictions[train_index], mlp_predictions[test_index]
        y_train_fold, y_test_fold = y_test[train_index], y_test[test_index]
        
        # Combine predictions for the fold
        combined_test = (cnn_weight * cnn_test) + (mlp_weight * mlp_test)
        final_test = (combined_test > 0.5).astype(int)
        fold_accuracy = accuracy_score(y_test_fold, final_test)
        fold_accuracies.append(fold_accuracy)
    
    # Average accuracy across folds
    mean_accuracy = np.mean(fold_accuracies)
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_weights = (cnn_weight, mlp_weight)

print(f"Best Weights from Cross-Validation: CNN={best_weights[0]}, MLP={best_weights[1]}")
print(f"Best Cross-Validated Accuracy: {best_accuracy:.4f}")


Best Weights from Cross-Validation: CNN=0.0, MLP=1.0
Best Cross-Validated Accuracy: 0.9700


In [103]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Prepare meta-model features
meta_features = np.column_stack((cnn_predictions, mlp_predictions))

# Define hyperparameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'solver': ['liblinear', 'lbfgs']
}

# Perform grid search
meta_model = LogisticRegression(random_state=42)
grid_search = GridSearchCV(meta_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(meta_features, y_test)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Meta-Model Accuracy: {grid_search.best_score_:.4f}")


Best Parameters: {'C': 1, 'solver': 'liblinear'}
Best Meta-Model Accuracy: 0.9689


## Try Grid Search on Everything

In [7]:
from tensorflow.keras.preprocessing import image
import numpy as np

def load_and_preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=target_size)
        img = image.img_to_array(img) / 255.0  # Normalize pixel values to range [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess the images
X_train_img_processed = load_and_preprocess_images(X_train_img)
X_test_img_processed = load_and_preprocess_images(X_test_img)


In [9]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

# CNN model creation function
def create_cnn_model(filters=32, kernel_size=(3, 3), dropout_rate=0.5, learning_rate=0.001):
    model = Sequential()
    model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters * 2, kernel_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

cnn_model = KerasClassifier(model=create_cnn_model, verbose=0)

# Define the parameter grid for CNN
cnn_param_grid = {
    'model__filters': [32, 64],
    'model__kernel_size': [(3, 3), (5, 5)],
    'model__dropout_rate': [0.3, 0.5],
    'model__learning_rate': [0.001, 0.0001],
    'batch_size': [32, 64],
    'epochs': [10, 15]
}

# Perform GridSearchCV for CNN
cnn_grid_search = GridSearchCV(estimator=cnn_model, param_grid=cnn_param_grid, cv=3, scoring='accuracy')
cnn_grid_search.fit(X_train_img_processed, y_train)

# Best CNN model parameters and accuracy
print("Best CNN Parameters:", cnn_grid_search.best_params_)
print("Best CNN Accuracy:", cnn_grid_search.best_score_)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential mode

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential mode

KeyboardInterrupt: 